In [1]:
#Locate Spark in Jupyter notebook using findspark
#!pip install findspark
import findspark
findspark.init()

In [2]:
#Import SparkContext, SparkConf, SparkConf
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
#Create a object of spark context
sc = SparkContext()

In [4]:
#Create object of spark session
spark = SparkSession.builder.getOrCreate()

In [5]:
#Alias name to spark session
ss = spark

In [7]:
#List to spark dataframe
l = [('Alice', 1)]
ss.createDataFrame(l).collect()
#ss.createDataFrame(l).show()

[Row(_1='Alice', _2=1)]

In [8]:
#Adding column heading to the Dataframe
#ss.createDataFrame(l, ['name', 'age']).collect()
ss.createDataFrame(l, ['name', 'age']).show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



In [9]:
#Convert an RDD to a spark Dataframe
rdd = sc.parallelize(l)
spark.createDataFrame(rdd).collect()
df = spark.createDataFrame(rdd, ['name', 'age'])
df.show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



In [10]:
#working with spark rows
from pyspark.sql import Row
Person = Row('name', 'age')
person = rdd.map(lambda r: Person(*r))
df2 = spark.createDataFrame(person)
df2.show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



In [11]:
#Create user defined schema for dataframe and creation of dataframe from the above rdd
from pyspark.sql.types import *
schema = StructType([
   StructField("name", StringType(), True),
   StructField("age", IntegerType(), True)])
df3 = spark.createDataFrame(rdd, schema)
df3.show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



In [12]:
#convert a pandas object to spark dataframe
ss.createDataFrame(df.toPandas()).show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



In [13]:
#Initialize Dataframe reader
dfr = ss.read

In [14]:
#Read csv file into a datframe object
df = dfr.csv("C:/Users/sgove/OneDrive/Desktop/all.csv", header=True, inferSchema=True)

In [15]:
#Save df in cache memory
df.cache()
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [16]:
#Count number of records in the dataframe
df.count()

1893059

In [17]:
#See column names of the dataframe
df.columns

['SYMBOL',
 'SERIES',
 'OPEN',
 'HIGH',
 'LOW',
 'CLOSE',
 'LAST',
 'PREVCLOSE',
 'TOTTRDQTY',
 'TOTTRDVAL',
 'TIMESTAMP',
 '_c11']

In [18]:
df.dtypes

[('SYMBOL', 'string'),
 ('SERIES', 'string'),
 ('OPEN', 'double'),
 ('HIGH', 'double'),
 ('LOW', 'double'),
 ('CLOSE', 'double'),
 ('LAST', 'double'),
 ('PREVCLOSE', 'double'),
 ('TOTTRDQTY', 'int'),
 ('TOTTRDVAL', 'double'),
 ('TIMESTAMP', 'string'),
 ('_c11', 'int')]

In [20]:
#Apply SQL like functions on spark dataframe with functional programming
df.select("OPEN","HIGH").show(5)

+------+-------+
|  OPEN|   HIGH|
+------+-------+
| 37.75|  37.75|
| 43.75|   45.3|
|3374.0|3439.95|
| 281.8| 294.45|
| 127.0|  132.0|
+------+-------+
only showing top 5 rows



In [21]:
#Using Aggregate functions on Spark Dataframe
df.agg({"OPEN":"max","CLOSE":"max","HIGH":"max","LOW":"max"}).show()
print()
df.agg({"OPEN":"min","CLOSE":"min","HIGH":"min","LOW":"min"}).show()
print()
df.agg({"OPEN":"mean","CLOSE":"mean","HIGH":"mean","LOW":"mean"}).show()

+---------+--------+----------+---------+
|max(HIGH)|max(LOW)|max(CLOSE)|max(OPEN)|
+---------+--------+----------+---------+
| 116490.0|115500.0|  115899.0| 116480.0|
+---------+--------+----------+---------+


+---------+--------+----------+---------+
|min(HIGH)|min(LOW)|min(CLOSE)|min(OPEN)|
+---------+--------+----------+---------+
|     0.05|    0.05|      0.05|     0.05|
+---------+--------+----------+---------+


+-----------------+------------------+------------------+-----------------+
|        avg(HIGH)|          avg(LOW)|        avg(CLOSE)|        avg(OPEN)|
+-----------------+------------------+------------------+-----------------+
|366.7326563885202|356.00841225223786|360.86674629263746|361.2689533078383|
+-----------------+------------------+------------------+-----------------+



In [22]:
#Using Where clause
df.where("SYMBOL == 'TCS' and TIMESTAMP like '%OCT-2012'").show()

+------+------+-------+-------+-------+-------+-------+---------+---------+---------------+-----------+-----+
|SYMBOL|SERIES|   OPEN|   HIGH|    LOW|  CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|      TOTTRDVAL|  TIMESTAMP| _c11|
+------+------+-------+-------+-------+-------+-------+---------+---------+---------------+-----------+-----+
|   TCS|    EQ| 1295.0| 1309.8|1293.05|1302.95| 1300.3|   1295.9|   897809|1.16901054195E9|01-OCT-2012|35001|
|   TCS|    EQ| 1301.0| 1325.0| 1301.0|1322.65| 1322.0|  1302.95|  1159115| 1.5272549353E9|03-OCT-2012|51635|
|   TCS|    EQ| 1320.0|1321.65|1308.65|1317.15| 1319.0|  1322.65|   659642|  8.675407259E8|04-OCT-2012|24351|
|   TCS|    EQ|1310.25|1319.35| 1055.0| 1304.0|1301.15|  1317.15|  1918055| 2.4666925933E9|05-OCT-2012|66953|
|   TCS|    EQ| 1310.0|1315.25|1285.45| 1292.7| 1288.0|   1304.0|   842704| 1.0942989384E9|08-OCT-2012|45834|
|   TCS|    EQ| 1297.9| 1298.8| 1285.0|1294.85| 1295.0|   1292.7|   645577| 8.3436310255E8|09-OCT-2012|27678|
|   TCS|  

In [23]:
#Correlation coeffienct
df.corr("OPEN","CLOSE")

0.9997560995948557

In [24]:
#Describe function dataframe
df.select("OPEN","CLOSE","HIGH","LOW").describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|              OPEN|             CLOSE|              HIGH|               LOW|
+-------+------------------+------------------+------------------+------------------+
|  count|           1893059|           1893059|           1893059|           1893059|
|   mean| 361.2689533078383|360.86674629263746| 366.7326563885202|356.00841225223786|
| stddev|1747.4329870382217|1743.8405876986792|1763.9932112529843|1728.5487526946383|
|    min|              0.05|              0.05|              0.05|              0.05|
|    max|          116480.0|          115899.0|          116490.0|          115500.0|
+-------+------------------+------------------+------------------+------------------+



In [25]:
#Group by 
df.groupBy("SYMBOL").max().show()

+----------+---------+---------+--------+----------+---------+--------------+--------------+-----------------+---------+
|    SYMBOL|max(OPEN)|max(HIGH)|max(LOW)|max(CLOSE)|max(LAST)|max(PREVCLOSE)|max(TOTTRDQTY)|   max(TOTTRDVAL)|max(_c11)|
+----------+---------+---------+--------+----------+---------+--------------+--------------+-----------------+---------+
|  GEODESIC|   136.95|   142.75|   134.0|     136.2|    136.4|         136.2|       7084723|    5.197036675E8|    37532|
|  GODREJCP|   1418.9|   1459.0| 1391.15|    1408.4|   1415.0|        1408.4|      10360791|  6.04854060625E9|   108267|
|       KGL|    38.35|    38.65|    37.1|     38.15|     38.4|         38.15|      77700950|  2.59301957275E9|    19179|
| KIRLOSENG|   333.55|   345.05|   325.8|     330.1|    330.0|         330.1|       2040712|   2.8565836865E8|     8727|
| KIRLOSIND|    742.0|    767.0|  728.65|    747.45|    759.0|        747.45|        197971|    7.325842715E7|     2892|
|  KSBPUMPS|   749.95|    780.0|

In [26]:
#Convert Spark dataframe to rdd
df.rdd.take(10)

[Row(SYMBOL='20MICRONS', SERIES='EQ', OPEN=37.75, HIGH=37.75, LOW=36.35, CLOSE=37.45, LAST=37.3, PREVCLOSE=37.15, TOTTRDQTY=38638, TOTTRDVAL=1420968.1, TIMESTAMP='01-APR-2011', _c11=None),
 Row(SYMBOL='3IINFOTECH', SERIES='EQ', OPEN=43.75, HIGH=45.3, LOW=43.75, CLOSE=44.9, LAST=44.8, PREVCLOSE=43.85, TOTTRDQTY=1239690, TOTTRDVAL=55311204.35, TIMESTAMP='01-APR-2011', _c11=None),
 Row(SYMBOL='3MINDIA', SERIES='EQ', OPEN=3374.0, HIGH=3439.95, LOW=3338.0, CLOSE=3397.5, LAST=3400.0, PREVCLOSE=3364.7, TOTTRDQTY=871, TOTTRDVAL=2941547.35, TIMESTAMP='01-APR-2011', _c11=None),
 Row(SYMBOL='A2ZMES', SERIES='EQ', OPEN=281.8, HIGH=294.45, LOW=279.8, CLOSE=289.2, LAST=287.2, PREVCLOSE=281.3, TOTTRDQTY=140643, TOTTRDVAL=40264075.5, TIMESTAMP='01-APR-2011', _c11=None),
 Row(SYMBOL='AARTIDRUGS', SERIES='EQ', OPEN=127.0, HIGH=132.0, LOW=126.55, CLOSE=131.3, LAST=130.6, PREVCLOSE=127.6, TOTTRDQTY=2972, TOTTRDVAL=384468.2, TIMESTAMP='01-APR-2011', _c11=None),
 Row(SYMBOL='AARTIIND', SERIES='EQ', OPEN=50.

In [29]:
#Taking Sample from the Spark DataFrame
df.sample(fraction=0.02, seed=3).count()

37616

In [30]:
#Applying Summary function on the field 
df.sample(fraction=0.02, seed=3).select("OPEN","HIGH","LOW","CLOSE").summary().show()

+-------+------------------+------------------+------------------+------------------+
|summary|              OPEN|              HIGH|               LOW|             CLOSE|
+-------+------------------+------------------+------------------+------------------+
|  count|             37616|             37616|             37616|             37616|
|   mean|365.03302078902584|370.62902993406794| 359.7573077945581|364.75487664823504|
| stddev|1743.9119761381596|1761.3487886875632|1731.5701384892911| 1746.406333053236|
|    min|              0.05|              0.05|              0.05|              0.05|
|    25%|             24.75|              25.4|             24.05|              24.6|
|    50%|              80.6|              82.7|             78.65|              80.4|
|    75%|             266.3|             272.0|            260.95|             265.8|
|    max|          113750.0|          113750.0|          113750.0|          113750.0|
+-------+------------------+------------------+-------

In [31]:
#Create a temporary view so that it can be used as a SQL table
df.createTempView("stocks")

In [32]:
ss.sql("select * from stocks where SYMBOL IN ('TCS','GEOMETRIC')").show()

+---------+------+-------+-------+-------+-------+-------+---------+---------+---------------+-----------+-----+
|   SYMBOL|SERIES|   OPEN|   HIGH|    LOW|  CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|      TOTTRDVAL|  TIMESTAMP| _c11|
+---------+------+-------+-------+-------+-------+-------+---------+---------+---------------+-----------+-----+
|GEOMETRIC|    EQ|  62.35|   64.5|   61.4|  63.25|  63.25|     61.3|    82246|     5179345.65|01-APR-2011| null|
|      TCS|    EQ| 1185.0|1198.75|1172.55|1180.15| 1181.9|   1183.9|   899812|1.06351115885E9|01-APR-2011| null|
|GEOMETRIC|    EQ|  100.7|  105.5|   99.1|  103.5| 102.55|    100.2|   124482|   1.27532668E7|01-APR-2013| 2690|
|      TCS|    EQ| 1565.0| 1573.7|1551.25|1556.85| 1552.1|  1575.75|   484406|  7.564599597E8|01-APR-2013|19638|
|GEOMETRIC|    EQ|  116.0|  121.0|  116.0|  120.0|  120.2|   115.55|   644060|     7.701543E7|01-APR-2014| 6430|
|      TCS|    EQ| 2145.0| 2185.0| 2144.9| 2176.7| 2175.4|  2133.15|  1377966| 2.9939665915E9|01

In [33]:
ss.sql("SELECT CORR(CLOSE,OPEN) AS PEARSON_CORR_CLOSE FROM stocks").show()

+------------------+
|PEARSON_CORR_CLOSE|
+------------------+
|0.9997560995948557|
+------------------+



In [34]:
#Save a Spark DataFrame as a CSV file on your System
a = ss.sql("select * from stocks where SYMBOL IN ('TCS','GEOMETRIC')")
a.write.option("header",True).csv("C:/Users/sgove/OneDrive/Desktop/sahbbir")